In [1]:
import numpy as np
import inspect
np.__version__

'1.21.6'

In [2]:
def retrieve_name(var):
  callers_local_vars = inspect.currentframe().f_back.f_locals.items()
  return [var_name for var_name, var_val in callers_local_vars if var_val is var]

In [3]:
def array_info(array, array_name="Noname"):
  print("*" * 60)
  print("<<<array_info func:{0}>>>".format(array_name))
  print("-" * 60)
  print("array=", array)
  print("type=", type(array))
  print("ndim:", array.ndim)
  print("shape:", array.shape)
  print("dtype:", array.dtype)
  print("size:", array.size)  
  print("itemsize:", array.itemsize) 
  print("nbytes:", array.nbytes)  
  print("strides:", array.strides) 
  print("*" * 60)

#NumPy view
Numpy 함수를 실행하는 동안 일부 함수는 입력 배열의 복사본을 반환하고 일부는 view를 반환합니다. 내용이 물리적으로 다른 위치에 저장되는 경우 이를 복사라고 합니다. 반면에 동일한 메모리 내용에 대해 다른 view가 제공되는 경우 이를 view라고 합니다.

##NumPy views: saving memory, leaking memory, and subtle bugs

Python의 NumPy 라이브러리를 사용하는 경우는 일반적으로 많은 메모리를 사용하는 큰 배열을 처리하기 때문입니다. 메모리 사용량을 줄이기 위해 불필요한 복사를 최소화하고 싶을 수 있습니다.

NumPy에는 memory view와 같은 많은 일반적인 경우에서 이를 투명하게 수행하는 기능이 내장되어 있습니다. 그러나 이 기능은 배열이 가비지 컬렉션되는 것을 방지하여 메모리 사용량을 높일 수도 있습니다. 그리고 어떤 경우에는 예상치 못한 방식으로 데이터가 변형되어 버그가 발생할 수 있습니다.

이러한 문제를 피하기 위해 view가 작동하는 방식과 코드에 미치는 영향을 알아보겠습니다.

###Preliminary: Python lists</br>
NumPy 배열과 view를 보기 전에 약간 유사한 데이터 구조인 Python의 list를 살펴보겠습니다.

NumPy 배열과 같은 Python list는 연속적인 메모리 chunk입니다. Python list를 슬라이싱하면 완전히 다른 list를 얻게 됩니다. 즉, 새로운 메모리 chunk를 할당한다는 의미입니다.

####Memory RSS
RSS는 Resident Set Size이며 해당 프로세스에 할당되고 RAM에있는 메모리 양을 표시하는 데 사용됩니다. 스왑 아웃 된 메모리는 포함되지 않습니다. 해당 라이브러리의 페이지가 실제로 메모리에있는 한 공유 라이브러리의 메모리가 포함됩니다.

In [ ]:
from psutil import Process
Process().memory_info().rss

94449664

In [ ]:
list1 = [None] * 1000_000

In [ ]:
Process().memory_info().rss

102244352

In [ ]:
list2 = list1[:500_000]

In [ ]:
Process().memory_info().rss

106434560

list를 슬라이싱하면 더 많은 메모리가 할당됩니다. 그리고 두 번째 list(list2)은 독립적인 copy본이기 때문에 변경해도 첫 번째 list(list1)에는 영향을 미치지 않습니다.

In [ ]:
list2[0] = "abc"
print(list2[0])

abc


In [ ]:
print(list1[0])

None


(일반적으로) 슬라이싱할 때 NumPy 배열은 복사되지 않습니다.
NumPy 배열은 다르게 작동합니다. 매우 큰 배열로 작업할 수 있다고 가정하기 때문에 대부분 작업들은 배열을 복사하지 않고 원래 배열이 가리키는 동일한 연속 메모리 청크에 대한 view를 제공합니다.

첫 번째 결과는 원래 배열에 대한 view일 뿐이므로 슬라이싱이 더 많은 메모리를 할당하지 않는다는 것입니다.

In [ ]:
arr = np.arange(0, 1_000_000)
Process().memory_info().rss

114835456

In [ ]:
view = arr[:500_000]
Process().memory_info().rss

114835456

view 객체는 500,000 바이트 사이즈의 배열처럼 보이며 int64 데이터타입의 배열인 경우 약 4MB의 메모리를 필요할 것입니다. 그러나 이는 동일한 원본 배열에 대한 view 객체에 대한 것이므로 추가 메모리가 할당된 것은 아닙니다

###Leaking memory with views</br>
view 사용의 한 가지 결과는 메모리를 절약하는 대신 메모리를 누수할 수 있다는 것입니다. 이는 view가 원래 배열이 Garbage Collection되는 것을 방지하기 때문입니다.

큰 배열의 작은 메모리 chunk만 사용하기로 결정했다고 가정해 보겠습니다.

In [ ]:
arr = np.arange(0, 100_000_000)
Process().memory_info().rss

915521536

In [ ]:
small_slice = arr[:10]
del arr
Process().memory_info().rss

1717489664

이것이 Python list인 경우 원본 객체를 삭제하면 메모리가 해제됩니다. 그러나 이 경우 배열에 대한 직접적인 참조가 없더라도 view는 여전히 있습니다. 즉, 메모리의 일부에만 참조가 있더라도 메모리가 해제되지 않습니다.

실제로 view를 통해 원본 배열에 액세스할 수 있습니다.

In [ ]:
small_slice

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
small_slice.base

array([       0,        1,        2, ..., 99999997, 99999998, 99999999])

결과적으로 모든 view를 삭제한 후에만 원래 배열의 메모리가 해제됩니다.

In [ ]:
del small_slice
Process().memory_info().rss

1717489664

In [ ]:
Process().memory_info().rss

1717489664

###Unexpected mutation
view의 또 다른 결과는 view를 수정하면 원래 배열이 변경된다는 것입니다. Python list의 경우 새 객체가 복사본이기 때문에 슬라이스 결과를 수정해도 원본 list가 수정되지 않는다는 점을 기억하십시오.

In [ ]:
l = [1, 2, 3]
l2 = l[:]
l2[0] = 17
l2

[17, 2, 3]

In [ ]:
l

[1, 2, 3]

NumPy view를 사용하면 view를 변경하면 원래 객체가 변경되며 둘 다 동일한 메모리를 가리킵니다.

In [ ]:
arr = np.array([1, 2, 3])
view = arr[:]
view[0] = 17
view

array([17,  2,  3])

In [ ]:
arr


array([17,  2,  3])

일부 NumPy API[함수]는 상황에 따라 view 또는 복사본을 반환할 수 있기 때문에 예기치 않은 상황이 발생할 가능성이 높아집니다. 예를 들어 일부 슬라이싱 결과는 view가 아닐 수 있습니다.

In [ ]:
arr = np.array([1, 2, 3])
arr2 = arr[:]
arr2.base is arr

True

In [ ]:
arr3 = arr[[True, False, True]] #fancy indexing은 사본을 생성한다
print(arr3)
arr3.base is arr

[1 3]


False

arr2를 변경하면 arr도 변경되지만 arr3을 변경해도 arr은 변경되지 않습니다.

In [ ]:
arr3[0] = 4
print(arr3)

[4 3]


In [ ]:
arr

array([1, 2, 3])

###Explicit copying with copy()</br>
원본 메모리를 참조하지 않으려면 명시적 복사를 통해 새 배열을 만들 수 있습니다. 이것은 예기치 않은 상황을 방지하는 데 유용할 수 있으며 원래 배열을 메모리에 유지하고 싶지 않은 경우에도 유용할 수 있습니다.

In [ ]:
arr = np.arange(0, 100_000_000)
Process().memory_info().rss

2518454272

In [ ]:
small_slice = arr[:10].copy()
del arr
Process().memory_info().rss

2518454272

In [ ]:
print(small_slice.base)

None


In [ ]:
Process().memory_info().rss

2518454272

In [ ]:
xrrs = np.arange(0, 100_000_000)
Process().memory_info().rss

2518528000

In [ ]:
xrrs_small_slice = xrrs[:10]
del xrrs
Process().memory_info().rss

2582671360

In [ ]:
del xrrs_small_slice
Process().memory_info().rss

2582683648

이 경우 small_slice는 view가 아니라 복사본이기 때문에 arr을 삭제하면 메모리가 확보됩니다.

###view를 효율적이고 안전하게 사용하기</br>
다양한 NumPy API에서 자동으로 view를 반환하므로 코드를 작성할 때 view를 고려해야 합니다.
* NumPy API가 view, copy 또는 둘 다를 반환하는지 문서에서 주의하십시오.
* 메모리에서 지우고 싶은 큰 배열이 있는 경우 해당 배열에 대한 직접적인 참조가 없을 뿐만 아니라 이를 참조하는 view도 없는지 확인하십시오.
* 배열을 변경하려는 경우 실제로 view이기 때문에 실수로 다른 배열을 변경하지 않는지 확인하십시오.
* view를 원하지 않으면 copy()방법을 사용하십시오.

##배열 할당과 검색


###복사(copy)와 뷰(view)이해하기

다차원 배열은 여러 차원의 데이터를 빠르게 연산할 수 있도록 처리됩니다.
이때 배열을 빨리 처리하기 위해 원본을 공유한 view나 새로운 사본이 만들어집니다.
보통 원본과 다른 배열을 추가적으로 만들 때에는 새롭게 배열을 생성하는 함수를 사용하고
별로도 사본을 만들 때는 copy 메소드를 사용해야 합니다.

###배열을 변수에 할당하기

In [ ]:
a = np.array([[1,2], [3,4]])
a

array([[1, 2],
       [3, 4]])

배열의 할당된 변수를 다른 변수에 할당할 수도 있습니다. 두 변수는 동일한 배열을 참조합니다. 이런 방식으로 처리하는 것은 별칭(alias)를 만드는 것입니다. 동일한 배열을 두 개의 변수에서 참조해서 사용한다는 뜻입니다.

In [ ]:
b = a
b

array([[1, 2],
       [3, 4]])

두 변수의 레퍼런스를 is 예약어로 비교하면 True이빈다. 별칭에서는 뷰를 만들지 않아서 base 속성에는 아무것도 없습니다. 그래서 이 속성과 원본 다차원 배열이 저장된 변수와의 레퍼런스 비교를 하면 Fasle입니다.

In [ ]:
a is b

True

In [ ]:
a is b.base

False

별칭은 원본과 동일하므로 변수 b에 첫 번째 인덱스의 원소를 갱신하면 원본 다차원 배열의 레퍼런스를 가지고 있어서 원본 다차원 배열을 갱신합니다.

In [ ]:
b[0] = 999

In [ ]:
a

array([[999, 999],
       [  3,   4]])

In [ ]:
b

array([[999, 999],
       [  3,   4]])

In [ ]:
a is b.base

False

###다차원 배열의 원본을 메모리에 공유하기
원본 배열을 가지고 새로운 사본 배열을 copy 메소드로 만들어서</br>
새로운 변수 e에 할당합니다. 새로운 배열이 만들어지면 뷰가 만들어진 것이 아니라서 base 속성에는 아무것도 없습니다.

In [ ]:
f = np.array([4,5,6,7,8])
e = f.copy()

In [ ]:
e.base is None

True


슬라이스 검색으로 만들어진 부분 배열은 원본 배열의 하나의 뷰를 제공합니다.</br>
이 슬라이스로 만들어진 부분 배열을 새로운 변수 g에 할당합니다. 이 변수 g에 저장된 배열은 뷰 이므로 원본 배열의 정보를 base 속성에 있으므로, 이를 조회하면 원본 배열을 출력합니다.</br>
이 base 속성에 있는 레퍼런스와 원본 배열이 저잗왼 변수 f의 레퍼런스를 is 예약어로 비교하면 동일한 레퍼런스라서 True를 표시합니다.

In [ ]:
g = f[:]

In [ ]:
g.base

array([4, 5, 6, 7, 8])

In [ ]:
g.base is f

True

뷰를 확인하는 다른 방법은 실제 사용하는 메모리에 대한 정보를 함수로 확인하는 것입니다.</br> 넘파이모듈에는 공유 메모리를 점검할 수 있는 함수 may_share_memory를 제공합니다.

In [ ]:
np.may_share_memory(g,f)

True

동일한 데이터를 공유해서 뷰로 만들어진 배열을 갱신하면 원본을 그대로 변경합니다.

In [ ]:
g[1] = 99999

In [ ]:
g

array([    4, 99999,     6,     7,     8])

In [ ]:
f

array([    4, 99999,     6,     7,     8])

뷰 배열을 직접 만들 수 있습니다. 이때는 view 메소드를 사용합니다. 하나의 배열의 뷰를 만들어서 새로운 변수 h에 할당합니다. 메모리 공유를 확인하면 위에서 처리한 슬라이스 검색과 동일한 결과가 나오는 것을 알 수 있습니다. 뷰가 만들어지면 메모리에 올라간 데이터를 공유하므로 실제 메모리 사용이 많지 않을 것을 알 수 있습니다. 새로운 배열을 만들지 않고 처리가 필요한 경우는 view 메소드를 사용해서 새로운 객체를 만들지만 데이터는 공유하는 방식을 적절하게 사용하면 좋습니다.

In [ ]:
h = f.view()

In [ ]:
h

array([    4, 99999,     6,     7,     8])

In [ ]:
h.base is f

True

In [ ]:
np.may_share_memory(h,f)

True

새로운 배열을 만드는 가장 간단한 방식은 array 함수에 copy 매개변수에 True로 지정해서 사용하는 것입니다. 이는 copy 메소드를 사용하는 것과 동일하게 항상 새로운 배열을 만듭니다</br>
새로운 배열이 만들어지면 이 배열의 데이터는 항상 메모리에 올라갑니다. 기존 배열의 뷰가 아니므로 base 속성에 아무것도 저장하지 않습니다.

In [ ]:
a = np.array([1,2,5,7])

In [ ]:
b = np.array(a, copy=True)

In [ ]:
b.base is None

True

In [ ]:
c = np.array(a)

In [ ]:
c.base is None

True